In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
pip install spacy

     ---------------------------------------- 12.1/12.1 MB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 50.1/50.1 kB 2.5 MB/s eta 0:00:00
     -------------------------------------- 122.2/122.2 kB 7.5 MB/s eta 0:00:00
     -------------------------------------- 481.9/481.9 kB 7.5 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 7.3 MB/s eta 0:00:00
     -------------------------------------- 181.6/181.6 kB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 45.9/45.9 kB ? eta 0:00:00
     ---------------------------------------- 6.6/6.6 MB 8.0 MB/s eta 0:00:00
     -------------------------------------- 45.0/45.0 kB 319.1 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
import spacy
le = LabelEncoder()


In [9]:
!python -m spacy download en_core_web_sm
!python -m textblob.download_corpora

     ---------------------------------------- 12.8/12.8 MB 7.5 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Shefali\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shefali\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shefali\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Shefali\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\Shefali\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Shefali\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already

In [10]:
# Load the dataset
df = pd.read_csv("news_dataset.csv")

df = df[:5000]

print(df['subject'].value_counts())

# Define the mapping dictionary and map the labels
mapping = {'fake': 0, 'true': 1}
df['label'] = df['label'].map(mapping)
df['subject'] = le.fit_transform(df['subject'])
# Separate features and labels
X = df.drop(columns=['date','label'])
y = df['label']

# Initialize SpaCy
nlp = spacy.load("en_core_web_sm")

politicsNews       1250
worldnews          1100
News               1049
politics            750
left-news           504
Government News     182
US_News              89
Middle-east          76
Name: subject, dtype: int64


In [11]:
df['subject'].value_counts()

6    1250
7    1100
2    1049
5     750
4     504
0     182
3      89
1      76
Name: subject, dtype: int64

In [12]:
# Define preprocessing function
def preprocess(text):
    doc = nlp(text)
    no_stop_words = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    processed_text = " ".join(no_stop_words)
    return processed_text

# Apply preprocessing to title and text columns
X['title'] = X['title'].apply(preprocess)
X['text'] = X['text'].apply(preprocess)

In [13]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['title'] + ' ' + X_train['text'])
X_train_sequences = tokenizer.texts_to_sequences(X_train['title'] + ' ' + X_train['text'])
X_test_sequences = tokenizer.texts_to_sequences(X_test['title'] + ' ' + X_test['text'])

# Padding sequences
max_sequence_length = max([len(sequence) for sequence in X_train_sequences])
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')


In [14]:
X_train

,title,text,subject
4227,Cleveland Bills Tamir Rice Family $ 500 UNBE...,November 22 2014 12 year old Tamir Rice gun Cl...,2
4676,India Modi fight protect home base election,new DELHI Reuters indian Prime Minister Narend...,7
800,teen vogue publish article teach Teen girl ana...,month 2016 election Teen Vogue feature persona...,4
3671,"EPA Pruitt take flight cost taxpayer $ 58,000 ...",Reuters U.S. Environmental Protection Agency E...,6
4193,syrian government say ready Moscow back congress,BEIRUT Reuters government Damascus ready congr...,7
...,...,...,...
4426,UNREAL PHOTO sum Anti trump liberal Wackos,great parenting huh,5
466,COMCAST GIVES EMPLOYEES Day PROTEST TRUMP Pro ...,wow get ready switch tv internet phone service...,4
3092,oh Brother White House compare Obama Pope,social justice warrior think know s good socia...,5
3772,House Russia probe put spotlight low profile l...,WASHINGTON Reuters Representative Mike Conaway...,6


In [15]:
# Define model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Bidirectional(LSTM(64)))  # Return sequences for the next LSTM layer
# model.add(Bidirectional(LSTM(32)))  # Bidirectional LSTM with 32 units
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train_padded, y_train, epochs=10, validation_split=0.1)

# Evaluate model
y_pred_prob = model.predict(X_test_padded)
y_pred = np.where(y_pred_prob > 0.5, 1, 0)  # 1 for true, 0 for fake

print(classification_report(y_test, y_pred, target_names=['fake', 'true']))


Epoch 1/10
113/113 [==============================] - 8166s 73s/step - loss: 0.2835 - accuracy: 0.8825 - val_loss: 0.0564 - val_accuracy: 0.9875
Epoch 2/10
113/113 [==============================] - 62309s 556s/step - loss: 0.0153 - accuracy: 0.9983 - val_loss: 0.0278 - val_accuracy: 0.9950
Epoch 3/10
113/113 [==============================] - 8959s 79s/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.0352 - val_accuracy: 0.9925
Epoch 4/10
113/113 [==============================] - 10100s 89s/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0354 - val_accuracy: 0.9900
Epoch 5/10
113/113 [==============================] - 9666s 86s/step - loss: 7.2405e-04 - accuracy: 1.0000 - val_loss: 0.0390 - val_accuracy: 0.9925
Epoch 6/10
113/113 [==============================] - 10790s 96s/step - loss: 5.1105e-04 - accuracy: 1.0000 - val_loss: 0.0475 - val_accuracy: 0.9900
Epoch 7/10
113/113 [==============================] - 58703s 523s/step - loss: 3.2093e-04 - accuracy: 1.0000 - val_los

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         3635100   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 3,719,709
Trainable params: 3,719,709
Non-trainable params: 0
_________________________________________________________________
